# Comments  

現在、新しい記事の著者はどのユーザーにでも設定できます。   
代わりに、自動的に現在のユーザーに設定する必要があります。   
既定のCreateViewは、私たちのために多くの機能を提供しますが、現在のユーザーを作成者に設定するには、カスタマイズする必要があります。   
作者はフィールドから削除され、form_validメソッドを使用して自動的に設定されます。

# Model

まず、既存のデータベースに「コメント」という別のテーブルを追加することができます。   
このモデルは、Articleと多対1の外部キーの関係を持つことになります。  
一つの記事は多くのコメントを持つことができますが、その逆はありません。   
伝統的には、外部キーフィールドの名前は単にそれがリンクするモデルなので、このフィールドはarticleと呼ばれます。   
他の2つのフィールドはコメントと著者になります。
articles / models.pyファイルを開き、既存のコードの下に以下を追加します。

-- code --

コメントモデルには__str__メソッドとget_absolute_urlメソッドがあり、主要な記事/ページに戻ります。   
モデルを更新したので、新しいマイグレーションファイルを作成して適用します。   
オプションである各コマンドの最後に記事を追加することで、ここで記事アプリを使用することを指定しています。   
これは良い習慣です。   
たとえば、2つの異なるアプリでモデルを変更した場合はどうなりますか？ アプリを指定しなかった場合、両方のアプリの変更は同じマイグレーションファイルに組み込まれ、将来はエラーをデバッグするのが難しくなります。   
各マイグレーションをできるだけ小さく小さくしてください。

-- cmd --

# Admin

新しいモデルを作った後は、実際のウェブサイトに表示する前に管理アプリでそれを再生するのは良いことです。   
それが見えるように、私たちのadmin.pyファイルにコメントを追加してください。

-- code --

次に、python manage.py runserverでサーバーを起動し、メインページhttp：// 127.0.0.1：8000 / admin /

私たちのアプリ「記事」の下に、コメントと記事の2つの表が表示されます。 コメントの横にある「+追加」をクリックします。 記事の下には、既存の記事のドロップダウン、著者と同じこと、コメントの横にテキストフィールドがあることがわかります。

記事を選択し、コメントを書いてから、あなたのスーパーユーザーではない作者を選んでください。 次に、[保存]ボタンをクリックします。

あなたは "コメント"ページであなたのコメントを次に見なければなりません。

この時点で、追加の管理フィールドを追加して、このページのコメントと記事を見ることができます。   
しかし、1つのPostモデルに関連するすべてのCommentモデルを見るだけでよいのではないでしょうか？   
私たちはDjangoの管理機能でインライン（Inlines）と呼ばれることができます。  
これは外部キーの関係をすばやく視覚的に表示します。   
使用される主なインラインビューには、TabularInlineとStackedInlineの2つがあります。   
両者の唯一の違いは、情報を表示するためのテンプレートです。   
TabularInlineでは、すべてのモデルフィールドが1行に表示され、StackedInlineでは各フィールドに独自の行があります。   
両方を実装して、どれを好むかを決めることができます。  
テキストエディタでarticles / admin.pyを次のように更新します。

-- code --

今すぐhttp：// 127.0.0.1：8000 / admin /のメイン管理ページに戻り、「記事」をクリックします。私のケースでは「4番目の記事」のコメントを追加した記事を選択します。
 
より良い、そう？
私たちは関連するすべての記事とコメントを1か所で確認して修正することができます。 デフォルトでは、Django管理者は3つの空行をここに表示します。 デフォルトの番号を変更することができます

余分なフィールドと一緒に表示されます。 したがって、デフォルトでフィールドを必要としない場合、コードは次のようになります。

-- code --

個人的には、TabularInlineを使用するほうが、より少ないスペースでより多くの情報を表示することができます。   
これに切り替えるには、CommentInlineをadmin.StackedInlineからadmin.TabularInlineに変更するだけです。

-- code --

管理ページを更新すると、新しい変更が表示されます。各モデルのすべてのフィールドが同じ行に表示されます。   
ずっといい。   
コメントを表示するためにテンプレートを更新する必要があります。

# Template

コメントは既存の記事アプリ内に存在するので、新しいコンテンツを表示するためにarticle_list.htmlとarticle_detail.htmlの既存のテンプレートを更新するだけで済みます。  
新しいテンプレートを作成したり、URLやビューを混乱させる必要はありません。  
我々がしたいことは、特定の記事に関連するすべてのコメントを表示することです。  
これは、データベースに特定の情報ビットを要求しているときに「クエリ」と呼ばれます。  
私たちのケースでは、外部キーを使って作業しています。  
それぞれの記事が関連するコメントモデルを検索するため、逆の関係に従います。   
Djangoには、FOO_setと呼ばれる「後方」関係のための組み込みの構文があります。  
ここで、FOOは小文字のソースモデル名です。  
したがって、Articleモデルでは、article_setを使用してモデルのすべてのインスタンスにアクセスできます。  
しかし、個人的に私はこの構文が紛らわしくて非直感的であることを強く嫌う。  
より良いアプローチはモデルにrelated_name属性を追加することです。  
代わりにこの逆の関係の名前を明示的に設定することができます。  
それをしましょう。  
コメントモデルにrelated_name属性を追加するには良いデフォルトは、それをForeignKeyを保持するモデルの複数の名前にすることです。

-- code --

データベースモデルを変更したばかりなので、移行ファイルを作成してデータベースを更新する必要があります。   
Control cを使用してローカルサーバーを停止し、次の2つのコマンドを実行します。   
まもなく使用するようにサーバーを再度スピンアップしてください。

-- cmd --

クエリーの理解にはしばらく時間がかかるので、リバースリレーションシップの考え方が混乱しているかどうかは気にしないでください。   
必要に応じてコードを実装する方法を説明します。   
そして、これらの基本的なケースを習得すれば、クエリーセットをどのようにフィルタリングするかを詳細に調べることができるので、必要な情報を正確に返すことができます。   
article_list.htmlファイルでは、コメントをカードフッターに追加できます。   
編集と削除のリンクを移動しました

カード本体に入れます。   
article.comments.allという各コメントにアクセスするには、まず記事モデルを見てから、コメントモデル全体の関連名前であるコメントを選択し、すべてのコメントを選択します。   
テンプレート内の外部キー・データを参照するために、この構文に慣れていくのに少し時間がかかることがあります。

記事ページをhttp：// 127.0.0.1：8000 / articles /に更新すると、新しいコメントがページに表示されます。 Yoohoo！ できます。 最初のメッセージの下にコメントが表示されています。

# Conclusion

より多くの時間でフォームに焦点を当てるので、ユーザーは記事/ページに直接新しい記事を書くことができ、コメントも追加することができます。  
しかし、この章の主な焦点は、Djangoで外部キー関係がどのように機能するかを示すことです。  
私たちの新聞アプリは完成しました。  
パスワードのリセットに電子メールを使用するなど、強力なユーザー認証フローを備えています。  

カスタムユーザーモデルを使用しているため、カスタムユーザーモデルにフィールドを追加する場合は、追加フィールドを追加するだけで簡単です。  
既定では現在0に設定されているすべてのユーザーの年齢フィールドが既に用意されています。  
必要に応じて、サインアップフォームに年齢ドロップダウンを追加し、13歳以上のユーザーのみにユーザーアクセスを制限することができます。  
または、65歳以上のユーザーに割引を提供することができます.CustomUserモデルに対して何をしたいかは、 。  
ウェブ開発の大部分は同じパターンを踏襲し、DjangoのようなWebフレームワークを使用することで、機能面で必要なものの99％が既に含まれているか、既存の機能をわずかにカスタマイズするだけです。